In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
pd.set_option('display.max_rows', 10)

In [4]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [5]:
cwd = os.getcwd()
print(cwd)
# cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/notebooks/AltModels
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git


In [6]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelperflat import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
from ogb.graphproppred import Evaluator

In [7]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 5
cuda:0, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, 

In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

5

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.11.0+cu115
CUDA: 11.5
3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]


### Preparing dataset 

In [13]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'loewe_thresh'
score_val = 1

In [14]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

data_fname = 'data_v4' # v2 for baseline models, v3 for additive samples, v4 for additive baseline

In [15]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
print(targetdata_dir)

path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/data/processed/DrugComb_loewe_thresh_1/data_v4


In [16]:
xFlat = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'X_flat.pkl'))
y = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'y.pkl'))
expression = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'expression.pkl'))

In [17]:
xFlatMat = torch.stack([torch.cat(i) for i in list(xFlat.values())])
xFlatMat.shape

torch.Size([418189, 18])

In [18]:
torch.tensor(expression)

tensor([[7.3013, 4.1251, 7.1277,  ..., 4.4361, 6.3370, 6.6896],
        [7.3013, 4.1251, 7.1277,  ..., 4.4361, 6.3370, 6.6896],
        [7.3013, 4.1251, 7.1277,  ..., 4.4361, 6.3370, 6.6896],
        ...,
        [7.1871, 3.5130, 7.6483,  ..., 3.6550, 6.8683, 6.7297],
        [7.1871, 3.5130, 7.6483,  ..., 3.6550, 6.8683, 6.7297],
        [7.1871, 3.5130, 7.6483,  ..., 3.6550, 6.8683, 6.7297]],
       dtype=torch.float64)

In [19]:
x = torch.cat([xFlatMat, torch.tensor(expression)], dim=1)
x.shape

torch.Size([418189, 926])

In [20]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
x.shape

torch.Size([418189, 926])

In [22]:
y.shape

(418189,)

In [23]:
fold_partitions = get_stratified_partitions(y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.9474584433484449
class: 1 norm count: 0.05254155665155516
validation data
class: 0 norm count: 0.9474533715925395
class: 1 norm count: 0.05254662840746054
test data
class: 0 norm count: 0.9474640713551257
class: 1 norm count: 0.05253592864487434

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.9474584433484449
class: 1 norm count: 0.05254155665155516
validation data
class: 0 norm count: 0.9474533715925395
class: 1 norm count: 0.05254662840746054
test data
class: 0 norm count: 0.9474640713551257
class: 1 norm count: 0.05253592864487434

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.9474617645593584
class: 1 norm count: 0.05253823544064166
validation data
class: 0 norm count: 0.9474533715925395
class: 1 norm count: 0.05254662840746054
test data
class: 0 norm count: 0.9474521150673139
class: 1 norm count: 0.0525478849326861

-------------------------
fold_num: 3
train data
class: 0 norm count: 

In [24]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 301095
Number of validation graphs: 33456
Number of testing graphs: 83638


### Preprocessing

In [25]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

In [26]:
x_np = x.numpy()
y_np = y

In [27]:
# pipeline = compose(scaler.fit_transform, np.tanh, scaler.fit_transform)

In [28]:
# x_np_norm = pipeline(x_np)

In [29]:
deepsynergy_input_size = x_np.shape[1]
deepsynergy_input_size

926

In [30]:
ids = list(range(len(y)))

In [31]:
dataset = TensorDataset(torch.tensor(x_np),torch.tensor(y), torch.tensor(ids))
dataset

In [32]:
x_np.shape

(418189, 926)

In [33]:
fold_partitions[0]['test']

array([     0,      8,      9, ..., 418170, 418172, 418180])

In [34]:
foldx = dataset[fold_partitions[0]['test']][0]
foldx

tensor([[6.3333, 0.0000, 2.6667,  ..., 4.4361, 6.3370, 6.6896],
        [5.2727, 0.0000, 3.0909,  ..., 4.4361, 6.3370, 6.6896],
        [6.2593, 0.0000, 2.8889,  ..., 4.4361, 6.3370, 6.6896],
        ...,
        [5.5152, 0.0000, 2.9394,  ..., 3.6550, 6.8683, 6.7297],
        [5.6296, 0.0000, 3.0000,  ..., 3.6550, 6.8683, 6.7297],
        [5.6842, 0.0000, 3.1053,  ..., 3.6550, 6.8683, 6.7297]],
       dtype=torch.float64)

### Deep Synergy

In [35]:
# training params total_thresh 4
tp = {
    "batch_size" : 300,
    "num_epochs" : 5,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-5, #3e-4
    "max_lr_mul": 5,
    "l2_reg" : 1e-5,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 8192,
    "exp_H2" : 4096
}

In [36]:
tp['deepsynergy_input_size'] = deepsynergy_input_size
tp['deepsynergy_input_size']

926

In [37]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition): #
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelperflat.run_exp_flat, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition)) #

# def create_q_process_attr(queue, x_np_norm, gpu_num, tphp, exp_dir, partition, labels): #
# #     fold_gpu_map = {0:gpu_num}
#     return mp.Process(target=deepadr.hyphelperflat.run_attribution, args=(queue, x_np_norm, gpu_num, tphp, exp_dir, partition, labels)) #

In [ ]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

# partition = fold_partitions[0]
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
#     device_gpu = get_device(True, index=q_i)
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    create_directory(os.path.join(exp_dir, "modelstates"))

#     tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
# for q_i in range(len(hyperparam_space)):
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
#     if(spawned_processes < len(hyperparam_space)):
# #         device_gpu = get_device(True, index=q_i)
#         time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#         exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp_"+str(q_i)+"_"+time_stamp))
#         tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

#         q_process = create_q_process(queue, used_dataset, released_gpu_num, tphp, exp_dir, partition)
#         q_processes.append(q_process)
#         spawn_q_process(q_process)
#         spawned_processes = spawned_processes + 1

Start: 2022-09-14_11-29-05
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_di

Iteration:  14%|█▎        | 136/1004 [00:03<00:19, 45.37it/s]

In [ ]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))